# Angular cross-spectra calculation

This notebook solves a line-of-sight integral of the form:
\begin{equation}
C^{XY}_\ell=\frac{1}{2}\int{\rm d}\chi\chi^{-2}K_X(\chi)K_{Y}(\chi)P_{q_\perp}\left(k=\frac{\ell}{\chi},z(\chi)\right),
\end{equation}

where
\begin{align}
    K_{\kappa_{\bf B}}(\chi) &= \frac{3}{2}H_0^2\Omega_ma^{-1}\int_0^{\chi}{\rm d}\chi'\frac{\chi'(\chi-\chi')}{\chi}\frac{{\rm d}\chi'}{{\rm d}z}p\left(z(\chi')\right),\\
    K_b(\chi) &= \frac{\sigma_{\rm T}\bar{n}_{e,0}}{c}a(\chi)^{-2}e^{-\tau},
\end{align}
are the kernels for the gravitomagnetic lensing convergence and kSZ effect, respectively.

Notice that the $C_\ell$ calculation is further simplified due to the simple choice of $p(z(\chi'))=\delta^{{\rm D}}(\chi'-\chi_s)$. Hence the lensing kernel becomes
\begin{align}
 K_{\kappa_{\bf B}}(\chi) &= \frac{3}{2}H_0^2\Omega_ma^{-1}\int_0^{\chi}{\rm d}\chi'\frac{\chi'(\chi-\chi')}{\chi}\frac{{\rm d}\chi'}{{\rm d}z}p\left(z(\chi')\right),\\
                          &= \frac{3}{2}H_0^2\Omega_ma^{-1}\frac{\chi_s(\chi-\chi_s)}{\chi}H^{-1}(\chi_s)
\end{align}

In [1]:
## Load dependencies
import numpy as np
import math

In [ ]:
from scipy.integrate import quad
from scipy.integrate import simpson as simps
from scipy.integrate import odeint
from scipy.interpolate import interp1d, UnivariateSpline, InterpolatedUnivariateSpline
from scipy.interpolate import interp2d

##############################################################
# Physical parameters:
##############################################################
sol = 3E5                      # speed of light in km/s
Omega_m= 0.3072
H0= 68.                        # km/s/Mpc
h = H0/100.

rho_c = 1.88E-26*h**2         # Critical density in [kg/m^3]
m_ele = 9.11E-31              # Mass of electron in [kg]
m_H   = 1.67E-27              # Mass of H-atom in [kg]
m_He  = 6.65E-27              # Mass of He-atom in [kg]
SigmaT= 6.65E-29              # Thompson scattering cross section in [m^2]
Ombh2 = 0.02224144
xe    = 1.                    # ionisation fraction - instantaneous reionization (xe=1 is fully ionized)
Yp    = 0.25                  # primordial helium fraction
tau_H = 0.07*(1-Yp)*Ombh2/h   # Thomson scattering optical depth
kSZ_bfac = tau_H*xe           # Prefactor for kSZ 'b' observable

##############################################################
# Some useful functions
##############################################################

def bin_data(data, bin_width):
    binned = data[:(data.size // width) * width].reshape(-1, width).mean(axis=1)
    return binned

def a_exp(z):
    a=1./(1.+z)
    return a

def Hubble(z):
#    H0/= 3.08568E19 # in 1/s
#    H0/=3E5         # 1/Mpc in c=1 units
    H = H0*np.sqrt(Omega_m*(1. + z)**3+(1. - Omega_m)) # H0 in km/s/Mpc
#    H/=(1+z)        # if Conformal Hubble is needed   
    return H

def f_lin(x):
    H = H0*np.sqrt(Omega_m/x**3+(1-Omega_m)) # H0 in km/s/Mpc
    Omega_ma = Omega_m/x**3*(H0/H)**2
    growth = Omega_ma**0.545
    return growth

def D_a(x):
    x_grid = np.linspace(1, x, 1000)
    Omega_m0 = 0.3072
    H0= 68.         # km/s/Mpc
    H = H0*np.sqrt(Omega_m0/x_grid**3+(1-Omega_m0))
    Omega_ma = Omega_m0/x_grid**3*(H0/H)**2
    D_int = (Omega_ma**0.545-1.)/x_grid
    D_exp = simps(D_int, dx=x_grid[1]-x_grid[0])
    D_a = x*np.exp(D_exp)
    return D_a

def dz_dr(z, r):
    # r = comoving distance
    dzdr = (Hubble(z)/sol/h)
    return dzdr

def dr_dz(r,z):
    # r = comoving distance in Mpc/h
    drdz = 1./(Hubble(z)/sol/h)
    return drdz

def r_of_z(z):
    # r = comoving distance in Mpc/h
    z_grid = np.linspace(0, z, 1000)                # r-bins between r=0 and rmax=r
    r_int = 1./(Hubble(z_grid)/sol/h)
    cdist = simps(r_int, dx=z_grid[1]-z_grid[0])
    return cdist

def n_ele(z):
    # Electron number density n_e = n_H + 2*n_He
    n_H  = (1-Yp)*((Ombh2/h**2)*rho_c)/m_H *(1+z)**3
    n_He =    Yp *((Ombh2/h**2)*rho_c)/m_He*(1+z)**3
    n_ele= n_H + 2*n_He
    return n_ele # [m^-3]

def tau_optical_depth(z):
    # [tau] = dimensionless
    # Integration in terms of dz=H*dr
    tau_optical_depth_int = lambda x: SigmaT*sol*n_ele(x)/(1+x)/(Hubble(x)) *3.086E22 #(Mpc/m factor)
    return quad (tau_optical_depth_int, 0, z)[0]  # [0]= output value

def tau_optical_depth_r(r):
    # [tau] = dimensionless
    # Integration in terms of dr
    tau_optical_depth_int = lambda x: SigmaT*n_ele(z_of_r(x)) *3.086E22 #(Mpc/m factor)
    return quad (tau_optical_depth_int, 0, r)[0]  # [0]= output value


# Vectorise function so it can take arrays as input
tau_optical_depth  =np.vectorize(tau_optical_depth)
tau_optical_depth_r=np.vectorize(tau_optical_depth_r)

##############################################################
# Instumental noise for lensing and CMB observables
##############################################################

def Noise_WL(ngal, sigma_e): 
    """
    Calculates the weak-lensing noise due to intrinsic ellipticity of galaxies
    """
    
    # [ngal] = arcmin^-2
    ngal*= 60.**2          # galaxies per deg^2
    ngal*=(180./np.pi)**2  # galaxies per rad^2
    N_WL= sigma_e**2/ngal
    return N_WL
    
def Noise_CMB(FWHM, sigma_pix, ell):
    """
    Calculates the instrumental noise of CMB experiments (from Dore et al 2003)
    """
    
    # [FWHM]      = arcmin
    # [sigma_pix] = \mu K
    
    sigma_pix/= 2.7E6                # Normalise by CMB temperature (in \mu K)
    FWHM/= 60.                       # arcmin to deg
    FWHM*= np.pi/180.                # deg to rad
    # Dore et al expression
    N_CMB = FWHM**2*sigma_pix**2/(4*np.pi)*np.exp( ell**2*FWHM**2/(8.*np.log(2)) )
    return N_CMB


def Cov_WL_x_CMB(C_ell_x, C_ell_WL, ngal, sigma_e, C_ell_CMB, FWHM, sigma_pix, ell_list):
    """
    Calculates the total Covariance for binned data of Lensing x CMB.
    See eq 50 of Dore et al 2004.
    """

    # Evaluate noise terms
    N_WL = Noise_WL (ngal, sigma_e)
    N_CMB= Noise_CMB(FWHM, sigma_pix, ell_list)

    # Add primary CMB (from CAMB)
    CMB_Cl_inter= interp1d(ls[1:], totCL[1:,0]*(2*np.pi/(ls[1:]*(ls[1:]+1)))  , kind='cubic')
    CMB_Pri_Cl= CMB_Cl_inter(ell_list)

    Cov_tot = (C_ell_WL + N_WL)*(C_ell_CMB+CMB_Pri_Cl + N_CMB) 
    Cov_tot+= C_ell_x**2
    Cov_tot/=(2*ell_list+1)
    return Cov_tot


In [3]:
# Calculation of redshifts for a set of comoving distances:
r_bins    = np.linspace(0, 4000, 40)
r_bins_int= np.linspace(0, 4000, 1000)

z_r = odeint(dz_dr, 0, r_bins_int)
z_list = z_r[:,0]

z_of_r = interp1d(r_bins_int, z_list)

----------------------------------
## Interpolate $P(k;z)$ along $k$ and $z$ directions
The z-interpolation requires the spectra at a set of redshifts.

Recall that we use:
- $P_{\delta}$
- $P_{\nabla\times{\bf q}}=k^2P_{{\bf q}_\perp}$

In [5]:
Pm_int_2d = interp1d(k_ps, Pk_ps, kind='cubic')
Pq_int_2d = interp1d(k_T, Pk_T  , kind='cubic')

### Load and interpolate the Momentum data in $z$

In [6]:
#######################################################
# Interpolation function for the B-potential P(k)
#######################################################
def PBB_z_int(k, z):    
    # Select neighbouring P(k,z) to interpolate
    if  (z<=0.5):
        z_l = 0.0
        z_r = 0.5
        Pk_l = Pk_Bvtot_z0
        Pk_r = Pk_Bvtot_z05
    elif(0.5<z<=1.0):
        z_l = 0.5
        z_r = 1.0
        Pk_l = Pk_Bvtot_z05
        Pk_r = Pk_Bvtot_z1
    elif(1.0<z<=1.5):
        z_l = 1.0
        z_r = 1.5
        Pk_l = Pk_Bvtot_z1
        Pk_r = Pk_Bvtot_z15
    elif(1.5<z<=2.0):
        z_l = 1.5
        z_r = 2.0
        Pk_l = Pk_Bvtot_z15
        Pk_r = Pk_Bvtot_z2
    elif(z>2.0):
        print('This value of z is out of range and cannot be interpolated.')
        return

    x  = (f_lin(1/(1+z  ))*D_a(1/(1+z  ))**1*Hubble(z  ))**2
    x_l= (f_lin(1/(1+z_l))*D_a(1/(1+z_l))**1*Hubble(z_l))**2
    x_r= (f_lin(1/(1+z_r))*D_a(1/(1+z_r))**1*Hubble(z_r))**2 
    
    # Interpolate z-range
    weight= ( x - x_l )/( x_r - x_l )
#    print(weight)
    PBB_z = Pk_l*(1 - weight) + Pk_r*weight
    PBB_z/= Momentum_eq_fac(z)**2            # Convert into momentum
    
    # Interpolate k-values
    PBB_int_z  = interp1d(k_PBB   , PBB_z , kind='cubic')
    PBB_int_zk = PBB_int_z(k)
    PBB_int_zk*= f_PBB_missing(k)    # Correct missing power due to large-scale cutoff
    
    return PBB_int_zk

# Vectorize function so it can take arrays as input
PBB_z_int = np.vectorize(PBB_z_int)

#######################################################
# Interpolation function for the Phi-potential P(k)
#######################################################

def PPhi_z_int(k, z):    
    # Select neighbouring P(k,z) to interpolate
    if  (z<=0.5):
        z_l = 0.0
        z_r = 0.5
        Pk_l = Pk_Phi_z0
        Pk_r = Pk_Phi_z05
    elif(0.5<z<=1.0):
        z_l = 0.5
        z_r = 1.0
        Pk_l = Pk_Phi_z05
        Pk_r = Pk_Phi_z1
    elif(1.0<z<=1.5):
        z_l = 1.0
        z_r = 1.5
        Pk_l = Pk_Phi_z1
        Pk_r = Pk_Phi_z15
    elif(1.5<z<=2.0):
        z_l = 1.5
        z_r = 2.0
        Pk_l = Pk_Phi_z15
        Pk_r = Pk_Phi_z2
    elif(z>2.0):
        print('This value of z is out of range and cannot be interpolated.')
        return
    
    x  = (D_a(1/(1+z  )))**2
    x_l= (D_a(1/(1+z_l)))**2
    x_r= (D_a(1/(1+z_r)))**2 
    
    # Interpolate z-range
    weight= ( x - x_l )/( x_r - x_l )
    Pk_z = Pk_l*(1 - weight) + Pk_r*weight
    
    # Interpolate k-values
    Pk_int_z  = interp1d(k_Phi_z0, Pk_z , kind='cubic')
    Pk_int_zk = Pk_int_z(k)
    
    return Pk_int_zk

# Vectorize function so it can take arrays as input
PPhi_z_int = np.vectorize(PPhi_z_int)

--------------------------------------
## Generalized function to calculate the $C^{XY}_\ell$
#### Can use the $P(k)$ and kernels for $\kappa_\Phi$, $\kappa_B$ or kSZ (b)

In [7]:
def C_ell_XY(rs, ell, kmin, kmax, case, Pk_evol=True):
    """
    Computes the C_ell for any relevant combination of observables.
    """

    rmin = 1E-2
    rr  = np.linspace(rmin, rs, 200)
    rrr = rr [np.where(ell/rr < kmax)[0]]
    rrr = rrr[np.where(ell/rrr> kmin)[0]]

    if(len(rrr)<2): return
    
    # Define Kernels:
    # Note: In all cases, the integration is performed w.r.t dr=dz/H
    #
    # Standard lensing kernel
    K_L =  (rrr*(rs-rrr)/rs)                  
    
    # Gravitomagnetic lensing kernel
    K_B =2*(rrr*(rs-rrr)/rs)                   
    
    # kSZ (b) kernel
    K_b = SigmaT*3.086E22/h*n_ele(z_of_r(rrr))/( 1+z_of_r(rrr))*np.exp(-tau_optical_depth(z_of_r(rrr)) )
           
    # Select case
    if(case==0):      # \kappa_B x kSZ (cross-spectrum)
        if(Pk_evol==False): 
            C_ell_int = PBB_z_int(ell/rrr, 0.0)/rrr**2
        else:
            C_ell_int = PBB_z_int(ell/rrr, z_of_r(rrr))/rrr**2
            
        C_ell_int*= K_B*K_b                    # Kernels        
        C_ell_int*= (ell/rrr)**4               # k^4 Factor to translate k^4*P_BB to P_qq
        C_ell_int*= 0.5                        # Include 1/2 overall factor in momentum P(k) derivations     
        
    elif(case==1): # \kappa_B
        if(Pk_evol==False): 
            C_ell_int = PBB_z_int(ell/rrr, 0.0)/rrr**2
        else:
            C_ell_int = PBB_z_int(ell/rrr, z_of_r(rrr))/rrr**2
            
        C_ell_int*= K_B**2                     # Kernels        
        C_ell_int*= (ell/rrr)**4               # k^4 Factor to translate k^4*P_BB to P_qq
        C_ell_int*= 0.5                        # Include 1/2 overall factor in momentum P(k) derivations
        
    elif(case==2): # kSZ
        if(Pk_evol==False): 
            C_ell_int = PBB_z_int(ell/rrr, 0.0)/rrr**2
        else:
            C_ell_int = PBB_z_int(ell/rrr, z_of_r(rrr))/rrr**2
            
        C_ell_int*= K_b**2                     # Kernels        
        C_ell_int*= (ell/rrr)**4               # k^4 Factor to translate k^4*P_BB to P_qq
        C_ell_int*= 0.5                        # Include 1/2 overall factor in momentum P(k) derivations     
        
    elif(case==3): # \kappa_Phi
        if(Pk_evol==False): 
            C_ell_int = PPhi_z_int(ell/rrr, 0.0)/rrr**2       
        else:
            C_ell_int = PPhi_z_int(ell/rrr, z_of_r(rrr))/rrr**2

        C_ell_int*= K_L**2                     # Kernels        
        C_ell_int*= (ell/rrr)**4               # Factor to translate k^4*P_Phi to P_den
        
    elif(case==-1): # ISW-\kappa_Phi cross-spectrum (spurious signal)
        C_ell_int = PK_CAMB    (z_of_r(rrr), ell/rrr)
        C_ell_int*= 9./4.*H0**4*Omega_m**2/sol**4/ell**2*(2.*rrr-rs)/rs*(1+z_of_r(rrr))**2 # Kernel        

    elif(case==-2): # REVISED ISW-\kappa_Phi cross-spectrum (spurious signal)
        C_ell_int = dPk_dz_oa2 (z_of_r(rrr), ell/rrr)
        C_ell_int*= 9./4.*H0**4*Omega_m**2/sol**4/ell**2*(rs-rrr)*rrr/rs # Kernel        
        C_ell_int*= Hubble(z_of_r(rrr))/sol     # Integration in terms of dr (comoving distance)
        
    # Now Integrate
    C_ell = simps(C_ell_int, dx=rrr[1]-rrr[0])
    return C_ell


# Wrapper to evaluate the integral for a set of multipoles
def C_ell_multipoles_XY(rs, ell_min, ell_max, ell_num, kmin, kmax, case, Pk_evol=True):
    """
    Wrapper function to evaluate the integral for a set of multipoles
    """

    ell_list = np.arange(ell_min_HR, ell_max_HR, 1)  # Sample modes linearly
    C_XY = np.zeros(len(ell_list))

    for i in range(len(ell_list)):
        C_XY[i] = C_ell_XY(rs, ell_list[i], kmin, kmax, case, Pk_evol)
         
    return C_XY

In [8]:
# Set multipoles range
ell_min_HR = 40
ell_max_HR = 1E4
ell_num_HR = 30

#Evaluate C^XY_\ell for the different cases
for i in range(len(R_snap_new[:-11])):
    C_bB_zi_evol = C_ell_multipoles_XY(R_snap_new[i], ell_min_HR, ell_max_HR, ell_num_HR, 5E-2, 15.0, 0) # b-B
    C_BB_zi_evol = C_ell_multipoles_XY(R_snap_new[i], ell_min_HR, ell_max_HR, ell_num_HR, 5E-2, 15.0, 1) # B-B
    C_bb_zi_evol = C_ell_multipoles_XY(R_snap_new[i], ell_min_HR, ell_max_HR, ell_num_HR, 5E-2, 15.0, 2) # b-b
    C_PP_zi_evol = C_ell_multipoles_XY(R_snap_new[i], ell_min_HR, ell_max_HR, ell_num_HR, 5E-2, 15.0, 3) # Phi-Phi
    C_IP_zi_evol = C_ell_multipoles_XY(R_snap_new[i], ell_min_HR, ell_max_HR, ell_num_HR, 5E-2, 15.0, -1)# ISW-Phi
    C_IP2_zi_evol = C_ell_multipoles_XY(R_snap_new[i], ell_min_HR, ell_max_HR, ell_num_HR, 5E-2, 15.0, -2)# REVISED ISW-Phi

    if(i==0):
        C_bB_z_evol     = np.zeros((1,len(C_bB_zi_evol)))
        C_BB_z_evol     = np.zeros((1,len(C_BB_zi_evol)))
        C_bb_z_evol     = np.zeros((1,len(C_bb_zi_evol)))
        C_PP_z_evol     = np.zeros((1,len(C_PP_zi_evol)))
        C_IP_z_evol     = np.zeros((1,len(C_IP_zi_evol)))
        C_IP2_z_evol     = np.zeros((1,len(C_IP2_zi_evol)))
        C_bB_z_evol[0,:]= C_bB_zi_evol    
        C_BB_z_evol[0,:]= C_BB_zi_evol    
        C_bb_z_evol[0,:]= C_bb_zi_evol    
        C_PP_z_evol[0,:]= C_PP_zi_evol    
        C_IP_z_evol[0,:]= C_IP_zi_evol    
        C_IP2_z_evol[0,:]= C_IP2_zi_evol    
    else:
        C_bB_z_evol     = np.append(C_bB_z_evol,[C_bB_zi_evol],axis=0)
        C_BB_z_evol     = np.append(C_BB_z_evol,[C_BB_zi_evol],axis=0)
        C_bb_z_evol     = np.append(C_bb_z_evol,[C_bb_zi_evol],axis=0)
        C_PP_z_evol     = np.append(C_PP_z_evol,[C_PP_zi_evol],axis=0)
        C_IP_z_evol     = np.append(C_IP_z_evol,[C_IP_zi_evol],axis=0)
        C_IP2_z_evol     = np.append(C_IP2_z_evol,[C_IP2_zi_evol],axis=0)

NameError: name 'R_snap_new' is not defined

## Sanity checks

In [ ]:
# Sanity checks
print(r_of_z(1), z_of_r(2300))

#K_b = (tau_H*xe*(1+z_of_r(r_bins))**2)*np.exp(-tau_optical_depth(z_of_r(r_bins)))
K_b = SigmaT*3.086E22/h*n_ele(z_of_r(r_bins))/(1+z_of_r(r_bins))*np.exp(-tau_optical_depth(z_of_r(r_bins)))

#plt.plot(z_of_r(r_bins), K_b)
#plt.plot(z_of_r(r_bins), np.exp(-tau_optical_depth(z_of_r(r_bins))))
#plt.plot(z_of_r(r_bins), tau_optical_depth  (z_of_r(r_bins)))
plt.plot(z_of_r(r_bins), n_ele(z_of_r(r_bins))/(1+z_of_r(r_bins)))
#plt.plot(z_of_r(r_bins), xe*tau_H*(1.+z_of_r(r_bins))**2/(Hubble(z_of_r(r_bins))/sol/h))